# Pregunta 1

In [1]:
# Simulación para estimar p_0 = P(N = 0) = e^(-λ) en distribución Poisson

set.seed(123) # Para reproducibilidad
N <- 3000 # Número de simulaciones

# Función para simular y calcular los tres estimadores
simular_estimadores <- function(lambda, n) {
  # Para almacenar resultados de las N simulaciones
  p_hat <- numeric(N)
  p_check <- numeric(N)
  p_tilde <- numeric(N)
  
  # Valor real de p_0 y valor de θ
  p_0 <- exp(-lambda)
  theta <- exp(n * lambda * (exp(-1/n) - 1))
  
  for (i in 1:N) {
    # Generar muestra aleatoria de Poisson
    x <- rpois(n, lambda)
    
    # Calcular estimador por método de momentos/máxima verosimilitud
    lambda_hat <- mean(x)
    p_hat[i] <- exp(-lambda_hat)
    
    # Calcular estimador basado en variables Bernoulli
    y <- as.numeric(x == 0)
    p_check[i] <- mean(y)
    
    # Calcular estimador Rao-Blackwell
    T_stat <- sum(x)
    p_tilde[i] <- (1 - 1/n)^T_stat
  }
  
  # Calcular promedios y errores estándar
  return(list(
    lambda = lambda,
    p_0 = p_0,
    theta = theta,
    n = n,
    p_hat_mean = mean(p_hat),
    p_hat_ee = sd(p_hat),
    p_check_mean = mean(p_check),
    p_check_ee = sd(p_check),
    p_tilde_mean = mean(p_tilde),
    p_tilde_ee = sd(p_tilde)
  ))
}

# Definir parámetros para la simulación
parametros <- expand.grid(
  lambda = c(1.0, 0.5, 2.0),
  n = c(25, 50, 80)
)

# Ejecutar simulaciones
resultados <- list()
for (i in 1:nrow(parametros)) {
  lambda <- parametros$lambda[i]
  n <- parametros$n[i]
  resultados[[i]] <- simular_estimadores(lambda, n)
}

# Convertir resultados a dataframe para visualización
tabla_resultados <- do.call(rbind, lapply(resultados, function(x) {
  data.frame(
    lambda = x$lambda,
    p_0 = x$p_0,
    theta = x$theta,
    n = x$n,
    p_hat = x$p_hat_mean,
    ee_p_hat = x$p_hat_ee,
    p_check = x$p_check_mean,
    ee_p_check = x$p_check_ee,
    p_tilde = x$p_tilde_mean,
    ee_p_tilde = x$p_tilde_ee
  )
}))

# Ordenar la tabla como en el enunciado
tabla_ordenada <- tabla_resultados[order(tabla_resultados$lambda, tabla_resultados$n), ]

# Mostrar tabla con formato
tabla_formato <- data.frame(
  lambda = tabla_ordenada$lambda,
  p_0 = round(tabla_ordenada$p_0, 4),
  theta = round(tabla_ordenada$theta, 4),
  n = tabla_ordenada$n,
  p_hat = round(tabla_ordenada$p_hat, 4),
  ee_p_hat = round(tabla_ordenada$ee_p_hat, 4),
  p_check = round(tabla_ordenada$p_check, 4),
  ee_p_check = round(tabla_ordenada$ee_p_check, 4),
  p_tilde = round(tabla_ordenada$p_tilde, 4),
  ee_p_tilde = round(tabla_ordenada$ee_p_tilde, 4)
)

print(tabla_formato)

  lambda    p_0  theta  n  p_hat ee_p_hat p_check ee_p_check p_tilde ee_p_tilde
1    0.5 0.6065 0.6125 25 0.6091   0.0836  0.6029     0.0965  0.6031     0.0844
2    0.5 0.6065 0.6096 50 0.6103   0.0604  0.6079     0.0681  0.6073     0.0607
3    0.5 0.6065 0.6084 80 0.6085   0.0478  0.6068     0.0546  0.6066     0.0479
4    1.0 0.3679 0.3752 25 0.3769   0.0741  0.3697     0.0970  0.3696     0.0741
5    1.0 0.3679 0.3716 50 0.3705   0.0519  0.3672     0.0684  0.3668     0.0519
6    1.0 0.3679 0.3702 80 0.3710   0.0409  0.3680     0.0537  0.3687     0.0409
7    2.0 0.1353 0.1408 25 0.1403   0.0391  0.1346     0.0691  0.1348     0.0384
8    2.0 0.1353 0.1381 50 0.1385   0.0276  0.1370     0.0490  0.1358     0.0273
9    2.0 0.1353 0.1370 80 0.1370   0.0220  0.1348     0.0382  0.1353     0.0218


# Pregunta 2

In [2]:
# Cálculo de intervalos de probabilidad para N(0,σ) y χ²ₙ

# Función para calcular intervalos de probabilidad N(0,σ)
calcular_intervalos_normal <- function(sigma, nivel_conf) {
  alpha <- 1 - nivel_conf
  
  # Intervalo con colas del mismo peso
  q_i_simetrico <- qnorm(alpha/2, mean = 0, sd = sigma)
  q_s_simetrico <- qnorm(1 - alpha/2, mean = 0, sd = sigma)
  w_simetrico <- q_s_simetrico - q_i_simetrico
  
  # Intervalo de amplitud mínima
  # Para una normal centrada en 0, el intervalo de amplitud mínima es simétrico
  # por lo que coincide con el de colas del mismo peso
  q_i_minimo <- q_i_simetrico
  q_s_minimo <- q_s_simetrico
  w_minimo <- w_simetrico
  
  return(c(nivel_conf, q_i_simetrico, q_s_simetrico, w_simetrico, 
           q_i_minimo, q_s_minimo, w_minimo))
}

# Función para encontrar el intervalo de amplitud mínima para χ²
encontrar_intervalo_minimo_chi2 <- function(df, nivel_conf) {
  alpha <- 1 - nivel_conf
  
  # Función objetivo: amplitud del intervalo
  f <- function(q_i) {
    q_s <- qchisq(pchisq(q_i, df) + nivel_conf, df)
    return(q_s - q_i)
  }
  
  # Encontrar límite inferior que minimiza la amplitud
  # Buscar en un rango razonable
  rango_busqueda <- seq(0.001, qchisq(0.9, df), length.out = 1000)
  amplitudes <- sapply(rango_busqueda, f)
  q_i_optimo <- rango_busqueda[which.min(amplitudes)]
  
  # Calcular límite superior correspondiente
  q_s_optimo <- qchisq(pchisq(q_i_optimo, df) + nivel_conf, df)
  
  # Amplitud del intervalo óptimo
  w_optimo <- q_s_optimo - q_i_optimo
  
  return(c(q_i_optimo, q_s_optimo, w_optimo))
}

# Función para calcular intervalos de probabilidad χ²ₙ
calcular_intervalos_chi2 <- function(df, nivel_conf) {
  alpha <- 1 - nivel_conf
  
  # Intervalo con colas del mismo peso
  q_i_simetrico <- qchisq(alpha/2, df)
  q_s_simetrico <- qchisq(1 - alpha/2, df)
  w_simetrico <- q_s_simetrico - q_i_simetrico
  
  # Intervalo de amplitud mínima
  resultado_minimo <- encontrar_intervalo_minimo_chi2(df, nivel_conf)
  q_i_minimo <- resultado_minimo[1]
  q_s_minimo <- resultado_minimo[2]
  w_minimo <- resultado_minimo[3]
  
  return(c(nivel_conf, q_i_simetrico, q_s_simetrico, w_simetrico, 
           q_i_minimo, q_s_minimo, w_minimo))
}

# Parámetros para la simulación
sigmas <- c(1.0, 2.0, 5.0)
niveles_conf <- c(0.90, 0.95, 0.99)
grados_libertad <- c(5, 10, 20)

# Calcular intervalos para distribución normal
resultados_normal <- list()
contador <- 1
for (sigma in sigmas) {
  for (nivel_conf in niveles_conf) {
    resultados_normal[[contador]] <- c(sigma, calcular_intervalos_normal(sigma, nivel_conf))
    contador <- contador + 1
  }
}

# Calcular intervalos para distribución chi-cuadrado
resultados_chi2 <- list()
contador <- 1
for (df in grados_libertad) {
  for (nivel_conf in niveles_conf) {
    resultados_chi2[[contador]] <- c(df, calcular_intervalos_chi2(df, nivel_conf))
    contador <- contador + 1
  }
}

# Convertir resultados a dataframes
tabla_normal <- do.call(rbind, resultados_normal)
colnames(tabla_normal) <- c("sigma", "1-alpha", "q_i_simetrico", "q_s_simetrico", "w_simetrico", 
                           "q_i_minimo", "q_s_minimo", "w_minimo")

tabla_chi2 <- do.call(rbind, resultados_chi2)
colnames(tabla_chi2) <- c("n", "1-alpha", "q_i_simetrico", "q_s_simetrico", "w_simetrico", 
                         "q_i_minimo", "q_s_minimo", "w_minimo")

# Mostrar resultados con formato
tabla_normal_formato <- data.frame(
  sigma = tabla_normal[, "sigma"],
  conf = tabla_normal[, "1-alpha"],
  q_i_sim = round(tabla_normal[, "q_i_simetrico"], 4),
  q_s_sim = round(tabla_normal[, "q_s_simetrico"], 4),
  w_sim = round(tabla_normal[, "w_simetrico"], 4),
  q_i_min = round(tabla_normal[, "q_i_minimo"], 4),
  q_s_min = round(tabla_normal[, "q_s_minimo"], 4),
  w_min = round(tabla_normal[, "w_minimo"], 4)
)

tabla_chi2_formato <- data.frame(
  n = tabla_chi2[, "n"],
  conf = tabla_chi2[, "1-alpha"],
  q_i_sim = round(tabla_chi2[, "q_i_simetrico"], 4),
  q_s_sim = round(tabla_chi2[, "q_s_simetrico"], 4),
  w_sim = round(tabla_chi2[, "w_simetrico"], 4),
  q_i_min = round(tabla_chi2[, "q_i_minimo"], 4),
  q_s_min = round(tabla_chi2[, "q_s_minimo"], 4),
  w_min = round(tabla_chi2[, "w_minimo"], 4)
)

print("Intervalos para distribución normal N(0,σ):")
print(tabla_normal_formato)

print("Intervalos para distribución chi-cuadrado χ²ₙ:")
print(tabla_chi2_formato)

Warning message in qchisq(pchisq(q_i, df) + nivel_conf, df):
"Se han producido NaNs"
Warning message in qchisq(pchisq(q_i, df) + nivel_conf, df):
"Se han producido NaNs"
Warning message in qchisq(pchisq(q_i, df) + nivel_conf, df):
"Se han producido NaNs"
Warning message in qchisq(pchisq(q_i, df) + nivel_conf, df):
"Se han producido NaNs"
Warning message in qchisq(pchisq(q_i, df) + nivel_conf, df):
"Se han producido NaNs"
Warning message in qchisq(pchisq(q_i, df) + nivel_conf, df):
"Se han producido NaNs"
Warning message in qchisq(pchisq(q_i, df) + nivel_conf, df):
"Se han producido NaNs"
Warning message in qchisq(pchisq(q_i, df) + nivel_conf, df):
"Se han producido NaNs"
Warning message in qchisq(pchisq(q_i, df) + nivel_conf, df):
"Se han producido NaNs"
Warning message in qchisq(pchisq(q_i, df) + nivel_conf, df):
"Se han producido NaNs"
Warning message in qchisq(pchisq(q_i, df) + nivel_conf, df):
"Se han producido NaNs"
Warning message in qchisq(pchisq(q_i, df) + nivel_conf, df):
"Se 

[1] "Intervalos para distribuci<U+00F3>n normal N(0,<U+03C3>):"
  sigma conf  q_i_sim q_s_sim   w_sim  q_i_min q_s_min   w_min
1     1 0.90  -1.6449  1.6449  3.2897  -1.6449  1.6449  3.2897
2     1 0.95  -1.9600  1.9600  3.9199  -1.9600  1.9600  3.9199
3     1 0.99  -2.5758  2.5758  5.1517  -2.5758  2.5758  5.1517
4     2 0.90  -3.2897  3.2897  6.5794  -3.2897  3.2897  6.5794
5     2 0.95  -3.9199  3.9199  7.8399  -3.9199  3.9199  7.8399
6     2 0.99  -5.1517  5.1517 10.3033  -5.1517  5.1517 10.3033
7     5 0.90  -8.2243  8.2243 16.4485  -8.2243  8.2243 16.4485
8     5 0.95  -9.7998  9.7998 19.5996  -9.7998  9.7998 19.5996
9     5 0.99 -12.8791 12.8791 25.7583 -12.8791 12.8791 25.7583
[1] "Intervalos para distribuci<U+00F3>n chi-cuadrado <U+03C7><U+00B2><U+2099>:"
   n conf q_i_sim q_s_sim   w_sim q_i_min q_s_min   w_min
1  5 0.90  1.1455 11.0705  9.9250  0.4725  9.4299  8.9575
2  5 0.95  0.8312 12.8325 12.0013  0.2968 11.1921 10.8952
3  5 0.99  0.4117 16.7496 16.3379  0.1027 15.1286 1

# Pregunta 3

In [3]:
# Análisis de tiempos de procesamiento de dos algoritmos

# Datos de los tiempos de procesamiento
algoritmo1 <- c(10.63, 10.96, 9.41, 10.90, 10.38, 9.50, 11.17, 9.74, 9.47, 8.97, 
                10.28, 10.08, 8.24, 11.10, 11.46, 8.17, 9.16, 10.58, 10.62, 8.98, 
                9.83, 8.90, 8.04, 11.18, 10.50)

algoritmo2 <- c(10.90, 9.93, 11.78, 11.52, 10.38, 8.65, 11.69, 10.10, 10.21, 9.07, 
                10.24, 10.50, 6.94, 11.29, 14.58, 9.00, 8.67, 12.87, 11.19, 8.78, 
                11.06, 10.27, 9.01, 11.41, 10.68)

# 1. Intervalo del 90% de confianza para μ₁ con σ₁ = 1
n1 <- length(algoritmo1)
media1 <- mean(algoritmo1)
sigma1 <- 1  # Desviación estándar conocida
alpha1 <- 0.10  # Nivel de significancia para 90% de confianza
z_crit1 <- qnorm(1 - alpha1/2)  # Valor crítico de la distribución normal

error_estandar1 <- sigma1 / sqrt(n1)
margen_error1 <- z_crit1 * error_estandar1

# Intervalo de confianza para μ₁
ic_inf1 <- media1 - margen_error1
ic_sup1 <- media1 + margen_error1

# 2. Intervalo del 95% de confianza para μ₂ con σ₂ desconocida
n2 <- length(algoritmo2)
media2 <- mean(algoritmo2)
s2 <- sd(algoritmo2)  # Desviación estándar muestral
alpha2 <- 0.05  # Nivel de significancia para 95% de confianza
t_crit2 <- qt(1 - alpha2/2, df = n2 - 1)  # Valor crítico de la distribución t

error_estandar2 <- s2 / sqrt(n2)
margen_error2 <- t_crit2 * error_estandar2

# Intervalo de confianza para μ₂
ic_inf2 <- media2 - margen_error2
ic_sup2 <- media2 + margen_error2

# 3. Intervalo de confianza del 95% para σ₁ y σ₂
s1 <- sd(algoritmo1)  # Desviación estándar muestral del algoritmo 1
alpha3 <- 0.05  # Nivel de significancia para 95% de confianza

# Para σ₁
chi_inf1 <- qchisq(1 - alpha3/2, df = n1 - 1)
chi_sup1 <- qchisq(alpha3/2, df = n1 - 1)

ic_inf_sigma1 <- sqrt((n1 - 1) * s1^2 / chi_inf1)
ic_sup_sigma1 <- sqrt((n1 - 1) * s1^2 / chi_sup1)

# Para σ₂
chi_inf2 <- qchisq(1 - alpha3/2, df = n2 - 1)
chi_sup2 <- qchisq(alpha3/2, df = n2 - 1)

ic_inf_sigma2 <- sqrt((n2 - 1) * s2^2 / chi_inf2)
ic_sup_sigma2 <- sqrt((n2 - 1) * s2^2 / chi_sup2)

# 4. Intervalo de confianza del 95% para el cociente de varianzas θ = σ²₁/σ²₂
var1 <- var(algoritmo1)
var2 <- var(algoritmo2)
cociente_varianzas <- var1 / var2

# Valores críticos de F
f_inf <- qf(alpha3/2, df1 = n1 - 1, df2 = n2 - 1)
f_sup <- qf(1 - alpha3/2, df1 = n1 - 1, df2 = n2 - 1)

# Intervalo de confianza para θ
ic_inf_theta <- cociente_varianzas / f_sup
ic_sup_theta <- cociente_varianzas / f_inf

# Mostrar resultados
cat("1. Intervalo del 90% de confianza para μ₁ (σ₁ = 1):\n")
cat("   [", round(ic_inf1, 4), ", ", round(ic_sup1, 4), "]\n\n")

cat("2. Intervalo del 95% de confianza para μ₂ (σ₂ desconocida):\n")
cat("   [", round(ic_inf2, 4), ", ", round(ic_sup2, 4), "]\n\n")

cat("3. Intervalos de confianza del 95% para las desviaciones estándar:\n")
cat("   σ₁: [", round(ic_inf_sigma1, 4), ", ", round(ic_sup_sigma1, 4), "]\n")
cat("   σ₂: [", round(ic_inf_sigma2, 4), ", ", round(ic_sup_sigma2, 4), "]\n\n")

cat("4. Intervalo de confianza del 95% para el cociente de varianzas θ = σ²₁/σ²₂:\n")
cat("   [", round(ic_inf_theta, 4), ", ", round(ic_sup_theta, 4), "]\n")

# Estadísticas descriptivas
cat("\nEstadísticas descriptivas:\n")
cat("Algoritmo 1:\n")
cat("   Media: ", round(media1, 4), "\n")
cat("   Desviación estándar: ", round(s1, 4), "\n")
cat("   Varianza: ", round(var1, 4), "\n\n")

cat("Algoritmo 2:\n")
cat("   Media: ", round(media2, 4), "\n")
cat("   Desviación estándar: ", round(s2, 4), "\n")
cat("   Varianza: ", round(var2, 4), "\n")

1. Intervalo del 90% de confianza para <U+03BC><U+2081> (<U+03C3><U+2081> = 1):
   [ 9.601 ,  10.259 ]

2. Intervalo del 95% de confianza para <U+03BC><U+2082> (<U+03C3><U+2082> desconocida):
   [ 9.7855 ,  11.0721 ]

3. Intervalos de confianza del 95% para las desviaciones est<U+00E1>ndar:
   <U+03C3><U+2081>: [ 0.7904 ,  1.4081 ]
   <U+03C3><U+2082>: [ 1.2169 ,  2.168 ]

4. Intervalo de confianza del 95% para el cociente de varianzas <U+03B8> = <U+03C3><U+00B2><U+2081>/<U+03C3><U+00B2><U+2082>:
   [ 0.1859 ,  0.9573 ]

Estad<U+00ED>sticas descriptivas:
Algoritmo 1:
   Media:  9.93 
   Desviaci<U+00F3>n est<U+00E1>ndar:  1.0122 
   Varianza:  1.0246 

Algoritmo 2:
   Media:  10.4288 
   Desviaci<U+00F3>n est<U+00E1>ndar:  1.5584 
   Varianza:  2.4287 
